Работу выполнил Данил Исламов (Stepik ID: 274397404)

<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления 

До этого в курсе не было речи про задачу ранжировaния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [1]:
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

--2021-02-28 10:50:58--  https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1453905423 (1.4G) [application/octet-stream]
Saving to: ‘SO_vectors_200.bin?download=1’

SO_vectors_200.bin? 100%[===================>]   1.35G  14.6MB/s    in 2m 1s   

2021-02-28 10:53:00 (11.5 MB/s) - ‘SO_vectors_200.bin?download=1’ saved [1453905423/1453905423]



In [2]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin?download=1", binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [3]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [4]:
print(f"Num of words: {len(wv_embeddings.index2word)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли слово `cat` топ-5 близких слов к слову `dog`? Какое место? 

In [5]:
for word, distance in wv_embeddings.most_similar(positive=['dog'], topn=5):
    print(word, distance, sep='\t')

animal	0.8564180135726929
dogs	0.7880867123603821
mammal	0.7623804807662964
cats	0.7621253728866577
animals	0.760793924331665


Ответ: Видно, что 'cat' **не** входит в пятёрку ближайших к 'dog' слов

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

Проверим, приведены ли слова в словаре к нижнему регистру (полагаю, что должны быть, просто на всякий случай) — если так, то токенизатор тоже должен будет приводить слова к нижнему регистру

In [6]:
not_lower = 0

for w in wv_embeddings.vocab.keys():
    if w.encode().isalpha() and not w.islower():
      print(w)
      not_lower += 1

print()
print(not_lower)


0


Слов, написанных латиницей и не приведённых к нижнему регистру нет $\implies$ токенизатор должен приводить слова к нижнему регистру

Далее реализован простой токенизатор и несколько его вариаций. Для нормализации слов в одной из них был выбран стемминг, т.к. лемматизация с использованием spaCy занимала значительно больше времени, а для общей (грубой) оценки эффекта нормализации мне показалось допустимым использовать только один из методов 

In [7]:
import numpy as np
import re
from nltk.tokenize import WordPunctTokenizer
from nltk.stem import SnowballStemmer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import spacy

# Простой самодельный токенизатор — достаёт слова и приводит их к нижнему 
# регистру. Далее буду ссылаться на него как на "простой токенизатор"
class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        splitted = re.findall('\w+', text)
        tokenized = [w.lower() for w in splitted]
        return tokenized

# Токенизатор, применяющий стемминг
class StemTokenizer(MyTokenizer):
    def __init__(self):
        super().__init__()
        self.stemmer = SnowballStemmer(language='english')

    def tokenize(self, text):
        splitted = re.findall('\w+', text)
        lowered = [w.lower() for w in splitted]
        tokenized = list(map(self.stemmer.stem, lowered))
        return tokenized

# Токенизатор, отбрасывающий стоп-слова
class StopTokenizer(MyTokenizer):
    def __init__(self):
        super().__init__()
        self.stopWords = set(stopwords.words('english'))
    
    def tokenize(self, text):
        splitted = re.findall('\w+', text)
        tokenized = [w.lower() for w in splitted if w not in self.stopWords]
        return tokenized

# Токенизатор, применяющий стемминг и отбрасывающий стоп-слова
class StopnStem(MyTokenizer):
    def __init__(self):
        super().__init__()
        self.stopWords = set(stopwords.words('english'))
        self.stemmer = SnowballStemmer(language='english')

    def tokenize(self, text):
        splitted = re.findall('\w+', text)
        lowered = [w.lower() for w in splitted if w not in self.stopWords]
        tokenized = list(map(self.stemmer.stem, lowered))
        return tokenized

# Токенизатор из nltk.tokenize; как следует из названия (и документации) —
# выделяет не только слова, но и пунктуацию, различные символы  
wp_tokenizer = WordPunctTokenizer()


my_tokenizer = MyTokenizer()
stem_tokenizer = StemTokenizer()
stop_tokenizer = StopTokenizer()
ss_tokenizer = StopnStem()

tokenizers = [my_tokenizer, stem_tokenizer, stop_tokenizer, ss_tokenizer, wp_tokenizer]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [8]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении
        
        return: векторное представление для вопроса
    """
    res = np.zeros(dim)
    tokens = tokenizer.tokenize(question)
    for token in tokens:
        if token in embeddings:
            res += embeddings[token]

    return res / len(tokens)


Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [9]:
# Используем простой токенизатор
ans = question_to_vec("I love neural networks", embeddings=wv_embeddings, tokenizer=my_tokenizer)[2]
print(np.around(ans, 2))

-0.96


### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями. 

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv 
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранг корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?

Ответ:

Hits@47 - DCG@1 $= (\frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le 47]) - (\frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le 1]) =$ $= \frac{1}{N} \sum_{i=1}^N\, ([rank\_q_i^{'} \le 47] - \frac{1}{\log_2(1+rank\_q_i^{'})} \cdot [rank\_q_i^{'} \le 1])$

В данном случае, поскольку ранг дубликата сравнивается  в Hits и DCG с разными позициями, разность можно максимизировать, если Hits достигнет своего максимума, а DCG, соответственно, минимума. Hits будет максимальной, когда ранги  всех дубликатов не больше 47. DCG неотрицательна, поэтому надо "согнать" её в 0. Этого можно добиться, если все ранги будут больше 1. Соответственно, если все ранги будут находиться в промежутке (1, 47], разность достигнет максимума и будет равна $ \frac{1}{N}\sum_{i=1}^N (1 - 0) = 1$

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера. 
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)





Ответ: 

DCG@10$(rank\_q_i^{'} = 9) = \frac{1}{\log_2(1+9)}\cdot[9 \le 10] = \frac{1}{\log_2{10}} = 0.3$

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [10]:
import numpy as np

In [12]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """
    hits_value = 0
    
    for r in dup_ranks:
      if r <= k:
          hits_value += 1
    
    return hits_value / len(dup_ranks)   

In [13]:
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
    """
    dcg_value = 0
    
    for r in dup_ranks:
      if r <= k:
          dcg_value += (1 / np.log2(1 + r))

    return dcg_value / len(dup_ranks)

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [14]:
import pandas as pd

In [15]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наги кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [i + 1 for j, _ in enumerate(copy_answers)
            for i, s in enumerate(candidates_ranking[j]) if s == copy_answers[j]]

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [16]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [17]:
# Спасибо автору статьи https://medium.com/@acpanjan/download-google-drive-files-using-wget-3c2c025a8b99
# за код для скачивания файлов с Google Drive :)
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_" -O stackoverflow_similar_questions.zip && rm -rf /tmp/cookies.txt

--2021-02-28 10:56:20--  https://docs.google.com/uc?export=download&confirm=KD42&id=1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_
Resolving docs.google.com (docs.google.com)... 173.194.217.102, 173.194.217.113, 173.194.217.139, ...
Connecting to docs.google.com (docs.google.com)|173.194.217.102|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0c-44-docs.googleusercontent.com/docs/securesc/i70bdv3op2vd8977dhtvju9p9nbas2qq/mqvelnff9c6ana72esbrkp8vqai6f4dv/1614509775000/09282986084580850099/17090548379999728213Z/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_?e=download [following]
--2021-02-28 10:56:20--  https://doc-0c-44-docs.googleusercontent.com/docs/securesc/i70bdv3op2vd8977dhtvju9p9nbas2qq/mqvelnff9c6ana72esbrkp8vqai6f4dv/1614509775000/09282986084580850099/17090548379999728213Z/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_?e=download
Resolving doc-0c-44-docs.googleusercontent.com (doc-0c-44-docs.googleusercontent.com)... 173.194.210.132, 2607:f8b0:400c:c0f::84
Conne

In [18]:
!unzip stackoverflow_similar_questions.zip

Archive:  stackoverflow_similar_questions.zip
   creating: data/
  inflating: data/.DS_Store          
   creating: __MACOSX/
   creating: __MACOSX/data/
  inflating: __MACOSX/data/._.DS_Store  
  inflating: data/train.tsv          
  inflating: data/validation.tsv     


Считайте данные.

In [19]:
import csv

*Небольшое лирическое отступление*

Для разделения данных на строки/предложения будем пользоваться самодельной функцией вместо готовой csv.reader. Причина в том, что при разбиении тренировочного датасета (в следующих пунктах задания) с помощью csv.reader строки получились разной длины, хотя в задании говорилось о том, что они состоят из двух предложений. Выведя минимум, среднее и максимум для длин всех строк я обнаружил, что самые короткие строки были длины 1, что было явно странно. Выяснилось, что эти строки были просто не разбиты (по неизвестной мне причине) — они содержали символы '\t' и, что более интересно, '\n', т.е. фактически некоторые строки оказались слеплены в одну, что могло навредить обучению. В валидационном датасете не было строк единичной длины, однако "слепленные" строки всё-таки нашлись — после применения функции, написанной ниже, количество строк увеличилось (по сравнению с количеством строк, полученным после разбиения с помощью csv.reader)

In [20]:
def read_corpus(filename):
    rows = []
    with open(filename, encoding='utf-8') as tsv_file:
        for line in tsv_file:
            rows += [[x] for x in line.split('\n') if x] 
        for i, r in enumerate(rows):
            rows[i] = [y for y in r[0].split('\t') if y]
    return rows

Нам понадобится только файл validation.

In [21]:
validation_data = read_corpus('./data/validation.tsv')

Посмотрим на минимальное, максимальное и среднее количество предложений в каждой строке (как я уже писал выше, это позволило заметить интересную аномалию на тренировочном датасете) 

In [22]:
ls = [len(s) for s in validation_data]
print(np.min(ls),
      np.mean(ls),
      np.max(ls), sep='\n')

289
1000.8106382978723
1001


Довольно интересно — среднее очень близко к максимальному значению. Поищем строки длиной менее 1001 предложения 

In [23]:
longs = sum([1 for s in validation_data if len(s) < 1001])
print(longs)

1


Строка, состоящая меньше, чем из 1001 предложения всего одна — выведем её

In [24]:
for i, s in enumerate(validation_data):
    if len(s) < 1001:
      print(i)
      print(s)

3759
['JQuery pass this to function', 'JQuery .slideDown() is sliding up', 'Create a file with data ur', 'Is there a way to have a Java8 duration of one year that accounts for leap years?', 'Easiest way to sort JSON.parse list in jQuery', "Change script tag type to 'text/ecmascript-6' or 'text/babel' when loading modules using requirejs", 'set Access database table field with form check box', 'webpack require non-js content in jest unit-tests', 'Binding Attached Behaviour from code', 'Getting nearby data from lat/lng in CakePHP', 'I keep getting exception sometimes on System.Threading.Tasks.TaskCompletionSource<bool> how can i solve it?', 'How do I view the SSIS packages in SQL Server Management Studio?', 'Android aar dependencies', 'Java RMI threads on client-side are executing sequentially or concurrently when calling the same remote object?', 'Three.js: add light to camera', 'Notes ACL Reader Level and Readers/Authors fields', 'Is there a way to make a branch invisible in TFS?', "A 

In [25]:
len(validation_data[3759])

289

Во всём датасете нашлась только одна строка немаксимальной длины. Ничего странного в ней я не заметил, так что выкидывать не буду

Кол-во строк

In [26]:
len(validation_data)

3760

Размер нескольких первых строк

In [27]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001


Посмотрим, как выглядит какая-нибудь случайная строка

In [28]:
import random
i = random.randint(0, len(validation_data))
print(validation_data[i])

['Back button exits application in android using appcelerator titanium', 'Clicking on android back button should show the last View. Titanium', 'How can i store 2 numbers in a 1 byte char?', 'Bloomberg API request limit', 'How to Send Periodic Ephemeral ("Hidden") Messages From a Slack Bot', 'How to filter verical html table with javascript', 'SSIS User Variable Assignment', 'Firebase in Cordova/Phonegap: Log in using Email/Password from within app?', 'How to register celery tasks across apps/projects in Django?', 'Cybersource undefined function cybs_load_config()', "Java - ImageIcon won't show image", 'ToggleSwitch color/styling', 'Android loading images to ListView AsyncTask', 'Is there a way to do command aliasing in matlab R2011b?', "Include 'ALL' option in dropdownlist bind using ViewBag", 'how to know the which archetype an existing maven project is built on?', 'Looping through AJAX and callbacks in jquery', 'Stata: dropping observations conditional on other, similar observations

### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [29]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [30]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    target = np.array([question_to_vec(question, embeddings, tokenizer)])
    variants = np.array([question_to_vec(c, embeddings, tokenizer) for c in candidates])
    dists = cosine_similarity(target, variants)[0]
    result = [(i, candidates[i]) for i, _ in enumerate(candidates)]
    result = sorted(result, key=lambda x: dists[x[0]], reverse=True)
    return result

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [31]:
questions = ['converting string to list', 'Sending array via Ajax fails'] 

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],
              
              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [32]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, my_tokenizer)
        for rank in ranks:
            print(rank)
        print()

(1, 'C# create cookie from string and send it')
(0, 'Convert Google results object (pure js) to Python object')
(2, 'How to use jQuery AJAX for an outside domain?')

(0, 'Getting all list items of an unordered list in PHP')
(2, 'select2 not displaying search results')
(1, 'WPF- How to update the changes in list item of a list')



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [ ]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(*, 'Getting all list items of an unordered list in PHP'), #скрыт
            (*, 'select2 not displaying search results'), #скрыт
            (*, 'WPF- How to update the changes in list item of a list')]] #скрыт

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

Ответ: 021

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [33]:
from tqdm.notebook import tqdm

In [ ]:
# Будем считать результаты сразу для всех токенизаторов
for t in tokenizers:
    wv_ranking = []
    max_validation_examples = 1000
    for i, line in enumerate(tqdm(validation_data)):
        if i == max_validation_examples:
            break
        q, *ex = line
        ranks = rank_candidates(q, ex, wv_embeddings, t)
        wv_ranking.append([r[0] for r in ranks].index(0) + 1)
    
    print(type(t).__name__)
    for k in [1, 5, 10, 100, 500, 1000]:
        print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k),
                                                  k, hits_count(wv_ranking, k)))
    print()   

MyTokenizer
DCG@   1: 0.415 | Hits@   1: 0.415
DCG@   5: 0.502 | Hits@   5: 0.582
DCG@  10: 0.525 | Hits@  10: 0.651
DCG@ 100: 0.570 | Hits@ 100: 0.874
DCG@ 500: 0.583 | Hits@ 500: 0.973
DCG@1000: 0.586 | Hits@1000: 1.000



StemTokenizer
DCG@   1: 0.332 | Hits@   1: 0.332
DCG@   5: 0.406 | Hits@   5: 0.474
DCG@  10: 0.427 | Hits@  10: 0.538
DCG@ 100: 0.479 | Hits@ 100: 0.795
DCG@ 500: 0.500 | Hits@ 500: 0.950
DCG@1000: 0.505 | Hits@1000: 1.000



StopTokenizer
DCG@   1: 0.416 | Hits@   1: 0.416
DCG@   5: 0.503 | Hits@   5: 0.582
DCG@  10: 0.525 | Hits@  10: 0.650
DCG@ 100: 0.571 | Hits@ 100: 0.874
DCG@ 500: 0.584 | Hits@ 500: 0.973
DCG@1000: 0.587 | Hits@1000: 1.000



StopnStem
DCG@   1: 0.329 | Hits@   1: 0.329
DCG@   5: 0.405 | Hits@   5: 0.474
DCG@  10: 0.426 | Hits@  10: 0.538
DCG@ 100: 0.479 | Hits@ 100: 0.799
DCG@ 500: 0.499 | Hits@ 500: 0.951
DCG@1000: 0.504 | Hits@1000: 1.000



WordPunctTokenizer
DCG@   1: 0.276 | Hits@   1: 0.276
DCG@   5: 0.332 | Hits@   5: 0.385
DCG@  10: 0.349 | Hits@  10: 0.438
DCG@ 100: 0.395 | Hits@ 100: 0.671
DCG@ 500: 0.421 | Hits@ 500: 0.875
DCG@1000: 0.434 | Hits@1000: 1.000



Видно, что "самодельный" токенизатор во всех реализациях показал себя лучше, чем WordPunctTokenizer. Этот факт кажется довольно просто объяснимым — как было сказано ранее, WordPunctTokenizer "из коробки" захватывает не только слова, но и пунктуацию и другие символы, что могло отрицательно повлиять на качество токенизации. Также примечательно, что использование стемминга ухудшало качество, а удаление стоп-слов почти не влияло на него. 


У меня есть 2 версии того, почему нормализация имела отрицательный эффект:

1.   Алгоритм стемминга был не приспособлен для данного типа текстов — всё же вопросы на stackoverflow значительно отличаются от обычных, скажем, художественных текстов — там может быть достаточно много терминов; коротких / сильно сокращённых слов; слов, содержащих символы, из-за чего их обрезание может привести к потере какой-либо важной информации

2.   Использованные эмбеддинги были обучены на целых, необрезанных словах, либо на словах, обрезанных другим методом, из-за чего нормализация приводила к несоответствию получаемых токенов имеющимся в словаре


Что касательно стоп-слов, предположу, что их либо не было среди эмбеддингов (тогда их вклад в вектор вопроса был бы нулевым), либо они имели там очень маленький вес (норму), что тоже кажется вполне естественным. При таком раскладе выглядит логичным, что удаление стоп-слов не вызвало критических изменений в качестве

  

In [ ]:
del wv_ranking

### Эмбеддинги, обученные на корпусе похожих вопросов




In [35]:
train_data = read_corpus('./data/train.tsv')

In [36]:
len(train_data)

1000000

Посмотрим на минимальное, максимальное и среднее количество предложений в каждой строке:

In [38]:
ls = [len(s) for s in train_data]
print(np.min(ls),
      np.mean(ls),
      np.max(ls), sep='\n')

2
2.256483
95


Посчитаем количество длинных строк:

In [39]:
longs = sum([1 for s in train_data if len(s) > 2])
print(longs)

192319


Посмотрим на некоторые строки, длина которых более 2 (заявленных) предложений, чтобы убедиться, что остальные предложения в них тоже соответствуют теме:

In [40]:
j = 0
for i, s in enumerate(train_data):
    if j == 9:
      break
    if len(s) > 2:
      print(i)
      print(s)
      print()
      j += 1

8
['Adding Prototype to JavaScript Object Literal', 'How does JavaScript .prototype work?', 'Javascript not setting property of undefined in prototyped object']

9
["What's the best way to get the directory from which an assembly is executing", 'What is the dependency inversion principle and why is it important?', 'Dependency Inversion with compile time configured Dependency Injection in an ASP.NET MVC 4 Solution']

12
['opaque element in a transparent in WPF', 'WPF transparency changing, when elements are bound', 'How do you override the opacity of a parent control in WPF?']

19
['How to change scroll bar position with CSS?', 'Scroll Bar at Top and Bottom in DataTables', 'horizontal scrollbar on top and bottom of table']

21
['private/public qt signals', 'Why do people use __(double underscore) so much in C++', 'Qt signals and slots: permissions']

23
['Injecting a Spring dependency into a JPA EntityListener', 'eventlisteners using hibernate 4.0 with spring 3.1.0.release?', 'Injecting

Среди выбранных длинных строк я не заметил каких-либо странностей, поэтому выкидывать ничего не будем

Посмотрим на случайную строку из train датасета

In [ ]:
i = random.randint(0, len(validation_data))
print(train_data[i])

['jQuery UI Dialog Buttons from variables', 'How set the names of jquery-ui buttons on dialog window from variable?']


Улучшите качество модели.<br>Склеим вопросы в строках и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

Посмотрим на среднюю длину склеенных предложений

In [41]:
np.mean([len((' '.join(s)).split(' ')) for s in train_data])

19.166858

Я обучал модели с разными значениями параметра window, чтобы проследить тенденцию в изменении качества. Результаты экспериментов приведены в следующем блоке. Для удобства код обучения с размером window, при котором качество получилось лучшим (в выборке размеров), будет вынесен отдельно, после упомянутого блока

##### Оценка моделей, обученных с разным размером window

In [42]:
from gensim.models import Word2Vec

In [ ]:
for t in tokenizers:
    
    words = [t.tokenize(' '.join(s)) for s in train_data]
    embeddings_trained = Word2Vec(words, # data for model to train on
                    size=200,                 # embedding vector size
                    min_count=5,             # consider words that occured at least 5 times
                     window=5).wv
    
    wv_ranking = []
    max_validation_examples = 1000
    for i, line in enumerate(tqdm(validation_data)):
        if i == max_validation_examples:
            break
        q, *ex = line
        ranks = rank_candidates(q, ex, embeddings_trained, t)
        wv_ranking.append([r[0] for r in ranks].index(0) + 1)
    
    print(type(t).__name__)
    for k in [1, 5, 10, 100, 500, 1000]:
        print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k),
                                                  k, hits_count(wv_ranking, k)))
    print()   

MyTokenizer
DCG@   1: 0.315 | Hits@   1: 0.315
DCG@   5: 0.391 | Hits@   5: 0.457
DCG@  10: 0.417 | Hits@  10: 0.538
DCG@ 100: 0.469 | Hits@ 100: 0.793
DCG@ 500: 0.489 | Hits@ 500: 0.945
DCG@1000: 0.495 | Hits@1000: 1.000



StemTokenizer
DCG@   1: 0.364 | Hits@   1: 0.364
DCG@   5: 0.432 | Hits@   5: 0.493
DCG@  10: 0.454 | Hits@  10: 0.563
DCG@ 100: 0.507 | Hits@ 100: 0.820
DCG@ 500: 0.525 | Hits@ 500: 0.958
DCG@1000: 0.529 | Hits@1000: 1.000



StopTokenizer
DCG@   1: 0.402 | Hits@   1: 0.402
DCG@   5: 0.495 | Hits@   5: 0.580
DCG@  10: 0.515 | Hits@  10: 0.641
DCG@ 100: 0.564 | Hits@ 100: 0.874
DCG@ 500: 0.576 | Hits@ 500: 0.970
DCG@1000: 0.579 | Hits@1000: 1.000



StopnStem
DCG@   1: 0.421 | Hits@   1: 0.421
DCG@   5: 0.509 | Hits@   5: 0.588
DCG@  10: 0.532 | Hits@  10: 0.659
DCG@ 100: 0.579 | Hits@ 100: 0.885
DCG@ 500: 0.590 | Hits@ 500: 0.974
DCG@1000: 0.593 | Hits@1000: 1.000



WordPunctTokenizer
DCG@   1: 0.254 | Hits@   1: 0.254
DCG@   5: 0.324 | Hits@   5: 0.387
DCG@  10: 0.341 | Hits@  10: 0.441
DCG@ 100: 0.397 | Hits@ 100: 0.719
DCG@ 500: 0.424 | Hits@ 500: 0.928
DCG@1000: 0.431 | Hits@1000: 1.000



In [ ]:
for t in tokenizers:
    
    words = [t.tokenize(' '.join(s)) for s in train_data]
    embeddings_trained = Word2Vec(words, # data for model to train on
                    size=200,                 # embedding vector size
                    min_count=5,             # consider words that occured at least 5 times
                     window=9).wv
    
    wv_ranking = []
    max_validation_examples = 1000
    for i, line in enumerate(tqdm(validation_data)):
        if i == max_validation_examples:
            break
        q, *ex = line
        ranks = rank_candidates(q, ex, embeddings_trained, t)
        wv_ranking.append([r[0] for r in ranks].index(0) + 1)
    
    print(type(t).__name__)
    for k in [1, 5, 10, 100, 500, 1000]:
        print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k),
                                                  k, hits_count(wv_ranking, k)))
    print()  

MyTokenizer
DCG@   1: 0.323 | Hits@   1: 0.323
DCG@   5: 0.403 | Hits@   5: 0.472
DCG@  10: 0.426 | Hits@  10: 0.541
DCG@ 100: 0.479 | Hits@ 100: 0.800
DCG@ 500: 0.498 | Hits@ 500: 0.946
DCG@1000: 0.503 | Hits@1000: 1.000



StemTokenizer
DCG@   1: 0.372 | Hits@   1: 0.372
DCG@   5: 0.442 | Hits@   5: 0.506
DCG@  10: 0.462 | Hits@  10: 0.569
DCG@ 100: 0.514 | Hits@ 100: 0.822
DCG@ 500: 0.533 | Hits@ 500: 0.962
DCG@1000: 0.537 | Hits@1000: 1.000



StopTokenizer
DCG@   1: 0.407 | Hits@   1: 0.407
DCG@   5: 0.504 | Hits@   5: 0.588
DCG@  10: 0.525 | Hits@  10: 0.655
DCG@ 100: 0.572 | Hits@ 100: 0.881
DCG@ 500: 0.584 | Hits@ 500: 0.972
DCG@1000: 0.587 | Hits@1000: 1.000



StopnStem
DCG@   1: 0.435 | Hits@   1: 0.435
DCG@   5: 0.520 | Hits@   5: 0.595
DCG@  10: 0.545 | Hits@  10: 0.673
DCG@ 100: 0.590 | Hits@ 100: 0.890
DCG@ 500: 0.601 | Hits@ 500: 0.975
DCG@1000: 0.604 | Hits@1000: 1.000



WordPunctTokenizer
DCG@   1: 0.273 | Hits@   1: 0.273
DCG@   5: 0.342 | Hits@   5: 0.405
DCG@  10: 0.360 | Hits@  10: 0.462
DCG@ 100: 0.415 | Hits@ 100: 0.732
DCG@ 500: 0.440 | Hits@ 500: 0.929
DCG@1000: 0.448 | Hits@1000: 1.000



In [ ]:
for t in tokenizers:
    
    words = [t.tokenize(' '.join(s)) for s in train_data]
    embeddings_trained = Word2Vec(words, # data for model to train on
                    size=200,                 # embedding vector size
                    min_count=5,             # consider words that occured at least 5 times
                     window=15).wv
    
    wv_ranking = []
    max_validation_examples = 1000
    for i, line in enumerate(tqdm(validation_data)):
        if i == max_validation_examples:
            break
        q, *ex = line
        ranks = rank_candidates(q, ex, embeddings_trained, t)
        wv_ranking.append([r[0] for r in ranks].index(0) + 1)
    
    print(type(t).__name__)
    for k in [1, 5, 10, 100, 500, 1000]:
        print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k),
                                                  k, hits_count(wv_ranking, k)))
    print()

MyTokenizer
DCG@   1: 0.342 | Hits@   1: 0.342
DCG@   5: 0.421 | Hits@   5: 0.493
DCG@  10: 0.442 | Hits@  10: 0.557
DCG@ 100: 0.493 | Hits@ 100: 0.807
DCG@ 500: 0.511 | Hits@ 500: 0.949
DCG@1000: 0.517 | Hits@1000: 1.000



StemTokenizer
DCG@   1: 0.370 | Hits@   1: 0.370
DCG@   5: 0.446 | Hits@   5: 0.517
DCG@  10: 0.466 | Hits@  10: 0.577
DCG@ 100: 0.515 | Hits@ 100: 0.817
DCG@ 500: 0.534 | Hits@ 500: 0.961
DCG@1000: 0.538 | Hits@1000: 1.000



StopTokenizer
DCG@   1: 0.422 | Hits@   1: 0.422
DCG@   5: 0.514 | Hits@   5: 0.595
DCG@  10: 0.537 | Hits@  10: 0.667
DCG@ 100: 0.582 | Hits@ 100: 0.881
DCG@ 500: 0.593 | Hits@ 500: 0.967
DCG@1000: 0.597 | Hits@1000: 1.000



StopnStem
DCG@   1: 0.443 | Hits@   1: 0.443
DCG@   5: 0.530 | Hits@   5: 0.606
DCG@  10: 0.552 | Hits@  10: 0.674
DCG@ 100: 0.599 | Hits@ 100: 0.896
DCG@ 500: 0.609 | Hits@ 500: 0.977
DCG@1000: 0.612 | Hits@1000: 1.000



WordPunctTokenizer
DCG@   1: 0.281 | Hits@   1: 0.281
DCG@   5: 0.348 | Hits@   5: 0.408
DCG@  10: 0.372 | Hits@  10: 0.481
DCG@ 100: 0.423 | Hits@ 100: 0.739
DCG@ 500: 0.448 | Hits@ 500: 0.932
DCG@1000: 0.455 | Hits@1000: 1.000



In [ ]:
for t in tokenizers:
    
    words = [t.tokenize(' '.join(s)) for s in train_data]
    embeddings_trained = Word2Vec(words, # data for model to train on
                    size=200,                 # embedding vector size
                    min_count=5,             # consider words that occured at least 5 times
                     window=19).wv
    
    wv_ranking = []
    max_validation_examples = 1000
    for i, line in enumerate(tqdm(validation_data)):
        if i == max_validation_examples:
            break
        q, *ex = line
        ranks = rank_candidates(q, ex, embeddings_trained, t)
        wv_ranking.append([r[0] for r in ranks].index(0) + 1)
    
    print(type(t).__name__)
    for k in [1, 5, 10, 100, 500, 1000]:
        print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k),
                                                  k, hits_count(wv_ranking, k)))
    print()

MyTokenizer
DCG@   1: 0.337 | Hits@   1: 0.337
DCG@   5: 0.419 | Hits@   5: 0.492
DCG@  10: 0.440 | Hits@  10: 0.558
DCG@ 100: 0.492 | Hits@ 100: 0.811
DCG@ 500: 0.509 | Hits@ 500: 0.948
DCG@1000: 0.515 | Hits@1000: 1.000



StemTokenizer
DCG@   1: 0.373 | Hits@   1: 0.373
DCG@   5: 0.449 | Hits@   5: 0.519
DCG@  10: 0.468 | Hits@  10: 0.577
DCG@ 100: 0.518 | Hits@ 100: 0.824
DCG@ 500: 0.536 | Hits@ 500: 0.957
DCG@1000: 0.540 | Hits@1000: 1.000



StopTokenizer
DCG@   1: 0.423 | Hits@   1: 0.423
DCG@   5: 0.515 | Hits@   5: 0.593
DCG@  10: 0.542 | Hits@  10: 0.675
DCG@ 100: 0.586 | Hits@ 100: 0.888
DCG@ 500: 0.596 | Hits@ 500: 0.971
DCG@1000: 0.599 | Hits@1000: 1.000



StopnStem
DCG@   1: 0.447 | Hits@   1: 0.447
DCG@   5: 0.535 | Hits@   5: 0.612
DCG@  10: 0.559 | Hits@  10: 0.685
DCG@ 100: 0.603 | Hits@ 100: 0.897
DCG@ 500: 0.613 | Hits@ 500: 0.978
DCG@1000: 0.616 | Hits@1000: 1.000



WordPunctTokenizer
DCG@   1: 0.286 | Hits@   1: 0.286
DCG@   5: 0.353 | Hits@   5: 0.414
DCG@  10: 0.374 | Hits@  10: 0.479
DCG@ 100: 0.426 | Hits@ 100: 0.735
DCG@ 500: 0.451 | Hits@ 500: 0.932
DCG@1000: 0.458 | Hits@1000: 1.000



##### Код обучения модели с размером window, при котором метрики были наибольшей величины (по сравнению с остальными экспериментами)

In [43]:
for t in tokenizers:
    
    words = [t.tokenize(' '.join(s)) for s in train_data]
    embeddings_trained = Word2Vec(words, # data for model to train on
                    size=200,                 # embedding vector size
                    min_count=5,             # consider words that occured at least 5 times
                     window=25).wv
    
    wv_ranking = []
    max_validation_examples = 1000
    for i, line in enumerate(tqdm(validation_data)):
        if i == max_validation_examples:
            break
        q, *ex = line
        ranks = rank_candidates(q, ex, embeddings_trained, t)
        wv_ranking.append([r[0] for r in ranks].index(0) + 1)
    
    print(type(t).__name__)
    for k in [1, 5, 10, 100, 500, 1000]:
        print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k),
                                                  k, hits_count(wv_ranking, k)))
    print()

MyTokenizer
DCG@   1: 0.355 | Hits@   1: 0.355
DCG@   5: 0.434 | Hits@   5: 0.507
DCG@  10: 0.455 | Hits@  10: 0.573
DCG@ 100: 0.502 | Hits@ 100: 0.807
DCG@ 500: 0.520 | Hits@ 500: 0.944
DCG@1000: 0.526 | Hits@1000: 1.000



StemTokenizer
DCG@   1: 0.385 | Hits@   1: 0.385
DCG@   5: 0.455 | Hits@   5: 0.521
DCG@  10: 0.475 | Hits@  10: 0.584
DCG@ 100: 0.524 | Hits@ 100: 0.823
DCG@ 500: 0.542 | Hits@ 500: 0.957
DCG@1000: 0.546 | Hits@1000: 1.000



StopTokenizer
DCG@   1: 0.436 | Hits@   1: 0.436
DCG@   5: 0.525 | Hits@   5: 0.604
DCG@  10: 0.549 | Hits@  10: 0.680
DCG@ 100: 0.591 | Hits@ 100: 0.881
DCG@ 500: 0.603 | Hits@ 500: 0.973
DCG@1000: 0.606 | Hits@1000: 1.000



StopnStem
DCG@   1: 0.454 | Hits@   1: 0.454
DCG@   5: 0.537 | Hits@   5: 0.609
DCG@  10: 0.562 | Hits@  10: 0.688
DCG@ 100: 0.606 | Hits@ 100: 0.894
DCG@ 500: 0.616 | Hits@ 500: 0.976
DCG@1000: 0.619 | Hits@1000: 1.000



WordPunctTokenizer
DCG@   1: 0.291 | Hits@   1: 0.291
DCG@   5: 0.360 | Hits@   5: 0.423
DCG@  10: 0.380 | Hits@  10: 0.484
DCG@ 100: 0.431 | Hits@ 100: 0.739
DCG@ 500: 0.456 | Hits@ 500: 0.931
DCG@1000: 0.463 | Hits@1000: 1.000



В полученных результатах можно заметить следующую закономерность: качество работы моделей росло по обеим метрикам  с увеличением размера window; в финальной версии использовался размер 25, что даже больше средней длины предложений из тренировочного датасета. 

Предположу, что это объясняется следующим: окна маленького размера способствуют выучиванию значений конкретного слова; при увеличении размеров окна в него попадает всё больше других слов предложения, из-за чего начинает больше учиться контекст, общая тема предложения, в котором используется заданный набор слов. Поскольку на stackoverflow, весьма вероятно, часто встречались специфические слова или термины (например, какой-нибудь 'object' или названия языков программирования), для определения схожести предложений было бы неплохо выучивать не использование конкретных слов, а контекст, задаваемый их наборами — как раз для такой цели и подходят сравнительно большие размеры окон.

Перейдём непосредственно к сравнению качества, показанного разными моделями.

Вновь все реализации простого токенизатора превзошли по качеству WordPunctTokenizer — предполагаемая причина этому уже была названа выше — WordPunctTokenizer "из коробки" отдельно захватывает пунктуацию и символы, что может негативно сказаться на качестве работы модели. 

Кроме того, можно заметить и кое-какие изменения по сравнению с качеством на предобученных эмбеддингах — теперь стемминг и, в особенности, удаление стоп-слов помогли значительно улучшить качество моделей. Возможно, это связано с тем, что в каждом отдельном случае эмбеддинги с тренировочного датасета были получены с использованием того же токенизатора, который впоследствии разбивал и валидационную часть. Таким образом, удаление стоп-слов позволило убрать значительную часть шума (если это можно так называть в контексте NLP), а стемминг, как и ожидалось, привёл разные формы одного слова к одной, позволив лучше улавливать контекст, что также внесло свой положительный вклад в итоговое качество. 

Отдельно отмечу, что использование токенизатора с применением стемминга и удалением стоп-слов в финальном варианте показало более высокие результаты, нежели простой токенизатор в случае предобученных эмбеддингов. Возможно, это служит аргументом в пользу того, что предобученные эмбеддинги (хоть, возможно, и были более качественными, нежели полученные при обучении) не были приспособлены для работы с получаемыми токенами — и поэтому, в том числе, стемминг ухудшал качество при работе с ними.

### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:


Подведём итог всем полученным выше промежуточным результатам.



При работе с предобученными эмбеддингами наилучшее качество показала модель, которая только находила слова (состоящие из букв латинского алфавита и цифр) и приводила их к нижнему регистру. 

При работе с обучаемыми эмбеддингами самое высокое качество, превзошедшее таковое на предобученных эмбеддингах, показала модель, в которой применялся стемминг и удалялись стоп-слова.

Нормализация и удаление стоп-слов оказали положительный эффект на качество работы модели с обучаемыми эмбеддингами, поскольку обучение проходило при помощи тех же токенизаторов, которыми позднее обрабатывались валидационные данные. При этом в случае с предобученными эмбеддингами стемминг "уронил" качество — вероятно, как раз из-за того, что получаемые после стемминга слова из предложений валидационного датасета далеко не всегда соответствовали таковым в имеющемся словаре.

Таким образом, судить о том, какие эмбеддинги лучше, сложно, но можно однозначно сказать, что при правильном наборе параметров обучаемые эмбеддинги показали себя несколько лучше предобученных (та же причина, что и выше — возможно, предобученные эмбеддинги не были приспособлены для работы с результатами конкретных токенизаторов).

Тем не менее, качество получилось довольно низким — ниже 0.5 на Hits@1 и DCG@1 — возможно, это вызвано тем, что токенизаторы были слишком грубы для данной задачи, использовали слишком общий подход — не учитывалась специфика местного словаря; возможная важность тех слов, которые могли быть выкинуты как стоп-слова. Также слова в вопросах, связанных с программированием, могут содержать различные символы, быть чувствительными к конкретному написанию. Возможно, стоило бы научить токенизаторы распознавать конкретные слова / комбинации символов, аббревиатуры и прочие особенности. Помимо этого, приведение к нижнему регистру может быть не везде уместно — в случае определённых названий, к примеру. Суммируя вышесказанное, если говорить про модификации токенизаторов, думаю, что для повышения качества модели их нужно лучше адаптировать для данной конкретной задачи. 

Кроме того, возможно, стоило каким-либо образом учитывать слова, которых не оказалось в словарях эмбеддингов — в использованном же варианте они просто выкидывались (~ вектор был нулевым), а это могло привести к потере информации. Также мне кажется несовершенным использование среднего векторов слов предложения в качестве вектора самого предложения, т.к. данный метод никак не учитывает порядок слов в предложении, что тоже бывает очень важно для понимания смысла. Словом, стоит поискать ещё какие-либо способы получения векторов различных предложений с учётом упомянутых проблем.


